In [0]:
%pip install -U openai

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached openai-1.54.3-py3-none-any.whl (389 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Using cached jiter-0.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (327 kB)
  Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
  Using cached httpcore-1.0.6-py3-none-any.whl (78 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-2c443105-b1c3-4d32-beb1-0112b338c518
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
  Attempting uninstall: openai
    Found existing installation: openai 0.28.1
    Not uninstalling openai at /databricks/python3/lib/python3.10/site-package

In [0]:
dbutils.library.restartPython()

In [0]:
with open("/Volumes/genai_demo/products/reports/Product-Report-Markdown.md", 'r') as file:
    markdown_text = file.read()

In [0]:
import re

# Extracting the Positive Feedback section using string slicing
start = markdown_text.find('Positive Feedback**')
end = markdown_text.find('Negative Feedback**')

# Extracting the content between these two sections
positive_feedback = markdown_text[start:end].strip()
clean_feedback = re.sub(r'\n|\*|_|-|`|#', '', positive_feedback)
clean_feedback

'Positive Feedback Excellent value for money Good quality and durable Easy to use and operate Great for beginners and advanced users Good for specific use cases (e.g., wildlife observation, target shooting) Nice design and aesthetics Effective and wonderful product'

In [0]:
from openai import OpenAI

#### PERFORM TEXT TRANSLATION
DATABRICKS_TOKEN = DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
serving_endpoint_url = dbutils.widgets.get("serving_endpoint_url")
t2t_model_name = dbutils.widgets.get("t2t_model_name")

def generate_prompt(system_text, user_text):
  # How to get your Databricks token: https://docs.databricks.com/en/dev-tools/auth/pat.html
  # DATABRICKS_TOKEN = os.environ.get('DATABRICKS_TOKEN')
  # Alternatively in a Databricks notebook you can use this:
  client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url=serving_endpoint_url
  )

  chat_completion = client.chat.completions.create(
    messages=[
    {
      "role": "system",
      "content": system_text
    },
    {
      "role": "user",
      "content": user_text
    }
    ],
    model=t2t_model_name,
    max_tokens=600
  )
  return chat_completion.choices[0].message.content

In [0]:
sys_text = "I sell cameras all around the world. My customers have left a bunch of feedback. Here is a summary of the positive reviews. Generate a prompt that I can feed my image generation model. Also, add some prompt for more artistic and innovative background. Your response should of the following format: prompt. Absolutely do not say anything other than the prompt."

img_prompt = generate_prompt(sys_text, clean_feedback)

In [0]:
image_gen_t2t_model_name = dbutils.widgets.get("image_gen_model_name")

def generate_image(prompt):
  client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url=serving_endpoint_url
  )

  images = client.images.generate(
    prompt=f"{img_prompt}",
    model=image_gen_t2t_model_name,
    n=3
  )
  return images

In [0]:
images = generate_image(img_prompt)


In [ ]:
%sql

USE CATALOG genai_demo;
USE SCHEMA products;
CREATE VOLUME IF NOT EXISTS social_media
COMMENT 'Volume for generated social_media content.';

In [0]:
import base64, random
for image in images.data:
  # Assuming images.data[1].b64_json contains your base64 encoded image data
  base64_data = image.b64_json

  # Decode the Base64 string
  image_data = base64.b64decode(base64_data)
  num = random.randint(1, 1000000)
  # Save the image to the volume - you can modify this path
  file_name = f"/Volumes/genai_demo/products/social_media/post_{num}.png"
  # Write the decoded image data to a file
  with open(file_name, "wb") as file:
      file.write(image_data)

  print("Image successfully written.")

Image successfully written.
Image successfully written.
Image successfully written.


In [0]:
sys_text = "I sell cameras all around the world. My customers have left a bunch of feedback. Here is a summary of the positive reviews. Generate a caption for a viral instagram and other social media posts using all the positive feedback. Say something funny and innovative. Make sure to use keywords from the reviews summary. Also include trending hashtags. Your response should be of the format: caption. Absolutely do not say anything else."

sm_caption = generate_prompt(sys_text, clean_feedback)

In [0]:
num = random.randint(1, 1000000)
with open(f'/Volumes/genai_demo/products/social_media/caption_text_{num}', 'w') as file:
  file.write(sm_caption)